# Data Preparation for the Nord_H2ub Spine Model

This jupyter notebook contains all routines for the preparation of the input data sources into a input data file for the model in Spine. 

**Authors:** Johannes Giehl (jfg.eco@cbs.dk), Dana Hentschel (djh.eco@cbs.dk), Lucia Ciprian (luc.eco@cbs.dk)

## General settings

### Packages:

In [5]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from datetime import timedelta
import math
import sys
import os
import pickle
import warnings
import re

### Methods:

In [7]:
# Import the data preparation functions
# Determine the current working directory
module_path = os.getcwd()

if os.path.basename(module_path) != '00_functions':
# Set the module path (adjust the relative path if necessary)
    module_path = os.path.abspath(os.path.join(module_path, '00_functions'))
    if module_path not in sys.path:
        sys.path.append(module_path)

# Load the functions and methods from the corresponding file
from nord_h2ub_data_preparation_functions import *
from nord_h2ub_data_preparation_main_functions import *
from nord_h2ub_data_preparation_helper_functions import *
from nord_h2ub_data_preparation_investment_functions import *

### Base parameters

In [65]:
#this should just be used when running it from the user interface
with open(os.path.join(module_path, 'parameters.pkl'), 'rb') as file:
    parameters = pickle.load(file)

In [69]:
if not parameters:  
    print("Warning: No parameters were injected. Using default parameters only.")
    # Optionally, you could provide some default initialization here.

(year, start_date, end_date, area, product, powers, powers_capacities, scenario, frequency, 
    model_name, temporal_block, stochastic_scenario, stochastic_structure, 
    report_name, reports, 
    electrolyzer_type, des_segments_electrolyzer, 
    share_of_dh_price_cap, price_level_power, power_price_variance, 
    roll_forward_use, roll_forward_size, num_slices, datetime_index, 
    candidate_nonzero, investment_period_default, 
    investment_cost_params,
    capacities_exisiting_params,
    capacity_ASU, 
    capacity_Electrolyzer, capacity_Haber_Bosch_reactor, capacity_Fischer_Tropsch_unit, capacity_RWGS_unit, capacity_Methanol_Plant,
    capacity_Electric_Steam_Boiler, capacity_anaerobic, capacity_biomethanation, capacity_co2_removal,
    investment_limit_params) = set_parameters(parameters)

### File paths

In [71]:
# Set path to correct folders
'''still not working if it is not started from this jupyter notebook'''
# Input data
excel_file_path = get_excel_file_path() + '/01_input_raw/'
# Prepared input data
output_file_path = get_excel_file_path() + '/02_input_prepared/'

In [73]:
# Set name of the relevant files

Model_structure_file = '/Model_Data_Base_' + product + '.xlsx'
efficiency_electrolyzer_file = '/Efficiency_Electrolyzers.xlsx'
distric_heating_price_file = 'energy_prices/district_heating_price_cap.xlsx'
investment_costs_file = '/investment_cost_overview/Investment_cost_overview.xlsx'
mapping_file = '/methanol_object_mapping.xlsx'

PV_data_availabilityfactors = 'PV_availability_factors_Kasso.xlsx'
Wind_data_availabilityfactors = 'Wind_availability_factors_Kasso.xlsx'
data_powerprices = 'Day_ahead_prices_' + str(year) + '.xlsx'
grid_costs = 'energy_prices/grid_costs.xlsx'

# Output file
output_file_name = product + '_Input_prepared.xlsx'
output_mapping_file_name = product + '_object_mapping.xlsx'


## Workflow of the data preparation

### General parameters

In [77]:
# Date index
date_index = pd.date_range(start=start_date, end=end_date, freq='h')
formatted_dates = date_index.strftime('%Y-%m-%dT%H:%M:%S')
df_formatted_dates = pd.DataFrame(formatted_dates, columns=['DateTime'])

df_time = pd.DataFrame(df_formatted_dates)

### Data import

In [80]:
warnings.filterwarnings("ignore", category=UserWarning, module="openpyxl")

# Model structure
df_model_units_raw = pd.read_excel(excel_file_path + product + Model_structure_file, sheet_name='Units', index_col=None)
df_model_connections_raw = pd.read_excel(excel_file_path + product + Model_structure_file, sheet_name='Connections', index_col=None)
df_model_storages_raw = pd.read_excel(excel_file_path + product + Model_structure_file, sheet_name='Storages', index_col=None)

# Variable efficiency
df_efficiency_electrolyzer = pd.read_excel(excel_file_path + product + efficiency_electrolyzer_file, sheet_name='Efficiency_'+electrolyzer_type)

# Availability factor
df_PV_availabilityfactors_values = pd.read_excel(excel_file_path+PV_data_availabilityfactors, skiprows=2, usecols=[0,1,2,3,4,5])
df_wind_availabilityfactors_values = pd.read_excel(excel_file_path+Wind_data_availabilityfactors, skiprows=2, usecols=[0,1,2,3])

# Power prices
df_powerprices_total_values = pd.read_excel(excel_file_path+data_powerprices)
# Only extracting the prices from our earlier defined area
df_powerprices_values = df_powerprices_total_values[df_powerprices_total_values['PriceArea'] == area]
df_powerprices_values = df_powerprices_values.reset_index(drop=True)
df_grid_costs = pd.read_excel(excel_file_path + grid_costs)

# District heating prices
df_district_heating_price = pd.read_excel(excel_file_path + distric_heating_price_file, sheet_name='Price_Cap_Calculation', index_col=None)

# Investment costs
df_investment_costs_raw = pd.read_excel(excel_file_path + investment_costs_file, sheet_name='Investment_Cost', index_col=None)

# Mapping between entity and parameter name
df_mapping = pd.read_excel(excel_file_path + product + mapping_file, sheet_name='Object_Mapping', index_col=None)

### Adjustments

#### Adjust base elements:

In [149]:
# Overwrite capacities with set ones from dropdowns
# Depends on what capacity is related to (input or output) -> needs to be done for products other than methanol!!!
for index, row in df_model_units_raw.iterrows():
    object_type = row['Object_type']
    if object_type in ['PEM_Electrolyzer', 'AEC_Electrolyzer', 'SOEC_Electrolyzer']:
        object_type = 'Electrolyzer'
    
    # Default capacity in/out: existing row
    
    # Check if external capacities given
    pattern = re.compile(rf"capacity_.*{object_type}")
    matching_vars = [var for var in globals() if pattern.match(var)]
    
    if matching_vars:
        variable_name = matching_vars[0]
        variable_value = globals()[variable_name]
        
        if variable_value is not None:
            if object_type in ['Electrolyzer', 'Electric_Steam_Boiler']:
                df_model_units_raw.at[index, 'Cap_Input1_existing'] = variable_value
            else: df_model_units_raw.at[index, 'Cap_Output1_existing'] = variable_value

In [151]:
# Overwrite power capacity value with value from powers_capacities set in dropdowns
# Add missing power source if necessary

# Step 1: Check and update for Solar plant
if 'Solar plant' in powers_capacities and powers_capacities['Solar plant'] is not None:
    if not (df_model_units_raw['Object_type'] == 'PV_plant').any():
        new_row = {col: np.nan for col in df_model_units_raw.columns}  # Initialize all columns with NaN
        new_row['Unit'] = 'PV_plant'
        new_row['Object_type'] = 'PV_plant'
        new_row['Output1'] = 'Power_Kasso'
        new_row['Cap_Output1_existing'] = powers_capacities['Solar plant']
        new_row_df = pd.DataFrame([new_row])
        df_model_units_raw = pd.concat([df_model_units_raw, new_row_df], ignore_index = True)
    else:
        df_model_units_raw.loc[df_model_units_raw['Object_type'] == 'PV_plant', 'Cap_Output1_existing'] = powers_capacities['Solar plant']

# Step 2: Check and update for Wind_onshore
if 'Wind onshore' in powers_capacities and pd.notna(powers_capacities['Wind onshore']):
    if not (df_model_units_raw['Object_type'] == 'Wind_onshore').any():
        new_row = {col: np.nan for col in df_model_units_raw.columns}  # Initialize all columns with NaN
        new_row['Unit'] = 'Wind_onshore'
        new_row['Object_type'] = 'Wind_onshore'
        new_row['Output1'] = 'Power_Kasso'
        new_row['Cap_Output1_existing'] = powers_capacities['Wind onshore']
        new_row_df = pd.DataFrame([new_row])
        df_model_units_raw = pd.concat([df_model_units_raw, new_row_df], ignore_index = True)
    else:
        df_model_units_raw.loc[df_model_units_raw['Object_type'] == 'Wind_onshore', 'Cap_Output1_existing'] = powers_capacities['Wind onshore']

# Step 3: Check and update for Wind_offshore
if 'Wind offshore' in powers_capacities and pd.notna(powers_capacities['Wind offshore']):
    if not (df_model_units_raw['Object_type'] == 'Wind_offshore').any():
        new_row = {col: np.nan for col in df_model_units_raw.columns}  # Initialize all columns with NaN
        new_row['Unit'] = 'Wind_offshore'
        new_row['Object_type'] = 'Wind_offshore'
        new_row['Output1'] = 'Power_Kasso'
        new_row['Cap_Output1_existing'] = powers_capacities['Wind offshore']
        new_row_df = pd.DataFrame([new_row])
        df_model_units_raw = pd.concat([df_model_units_raw, new_row_df], ignore_index = True)
    else:
        df_model_units_raw.loc[df_model_units_raw['Object_type'] == 'Wind_offshore', 'Cap_Output1_existing'] = powers_capacities['Wind offshore']

In [153]:
# Drop some unnecessary information
df_model_units = df_model_units_raw.drop(columns = ['Object_type']).copy()
df_model_connections = df_model_connections_raw.drop(columns = ['Object_type']).copy()
df_model_storages = df_model_storages_raw.drop(columns = ['Object_type']).copy()

# Create mapping tables for object name to type
df_model_units_mapping = df_model_units_raw[['Unit', 'Object_type']].copy()
df_model_connections_mapping = df_model_connections_raw[['Connection', 'Object_type']].copy()
df_model_storages_mapping = df_model_storages_raw[['Storage', 'Object_type']].copy()

df_model_units_mapping.rename(columns={'Unit': 'Object_name'}, inplace=True)
df_model_connections_mapping.rename(columns={'Connection': 'Object_name'}, inplace=True)
df_model_storages_mapping.rename(columns={'Storage': 'Object_name'}, inplace=True)

# Create a dataframe with mapping of all object in the model
df_model_object_mapping = pd.concat([df_model_units_mapping, df_model_connections_mapping, df_model_storages_mapping], axis=0)
df_model_object_mapping = df_model_object_mapping.reset_index(drop=True)

In [228]:
#transform the investment information from the user interface into a dataframe
df_filtered_investment_cost = filter_investment_data(name_parameter='investment_cost', **investment_cost_params)
df_filtered_investment_limit = filter_investment_data(name_parameter='investment_limit', **investment_limit_params)
df_filtered_existing_cap = filter_investment_data(name_parameter='capacities_exisiting', **capacities_exisiting_params)

In [250]:
#allign information of the dataframes for investments
df_investment_cost = map_parameters_by_similarity(df_model_object_mapping, df_filtered_investment_cost)
df_investment_limit = map_parameters_by_similarity(df_model_object_mapping, df_filtered_investment_limit)
df_existing_cap = map_parameters_by_similarity(df_model_object_mapping, df_filtered_existing_cap)
#move Object_name column to first column 
df_investment_limit = move_column_to_first(df_investment_limit, 'Object_name')
df_investment_cost = move_column_to_first(df_investment_cost, 'Object_name')
df_existing_cap = move_column_to_first(df_existing_cap, 'Object_name')
#merge the two dataframes to have one dataframe with all the investment information from the user interface
df_investment_params = pd.merge(df_investment_cost, df_investment_limit, on='Object_name', how='left')
df_investment_params = pd.merge(df_investment_params, df_existing_cap, on='Object_name', how='left')

In [157]:
df_units = process_dataframe(df_model_units, 'Unit', 'unit')
df_connections = process_dataframe(df_model_connections, 'Connection', 'connection')

In [159]:
# Adjust the storage loss rate values to fit to the SpineOpt implementation
df_model_storages = adjust_frac_state_loss(df_model_storages, 'frac_state_loss')

In [161]:
# Define the elements of the network
df_definition, df_nodes = create_definition_dataframe(df_model_units, df_model_connections)

In [163]:
# Create a data frame for all parameters of units

# Add fixed operation and maintenance cost
unit_fom_cost_df = create_object_parameters(df_model_units, 'Unit', 'fom_cost')
# Add unit minimal downtime
unit_min_down_time_df = create_object_parameters(df_model_units, 'Unit', 'min_down_time')
# Add unit on cost
unit_on_cost_df = create_object_parameters(df_model_units, 'Unit', 'units_on_cost')
# Add start up costs
start_up_cost_df = create_object_parameters(df_model_units, 'Unit', 'start_up_cost')
# Add shut down costs
shut_down_cost_df = create_object_parameters(df_model_units, 'Unit', 'shut_down_cost')

connection_fom_cost_df = create_object_parameters(df_model_connections, 'Connection', 'fom_cost')

# Create a complete data frame with all parameters
unit_parameters_df = pd.concat([unit_fom_cost_df, unit_min_down_time_df, unit_on_cost_df, start_up_cost_df, shut_down_cost_df, connection_fom_cost_df], ignore_index=True)

# Show table head for control
unit_parameters_df.head()

C:\Users\jfg.eco\Documents\GitHub\Nord_H2ub\spine_projects\01_input_data\00_functions\nord_h2ub_data_preparation_functions.py:388: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unit_parameter_df = pd.concat([unit_parameter_df, pd.DataFrame([new_row])], ignore_index=True)
C:\Users\jfg.eco\Documents\GitHub\Nord_H2ub\spine_projects\01_input_data\00_functions\nord_h2ub_data_preparation_functions.py:388: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  unit_parameter_df = pd.concat([unit_parameter_df, p

,Object_name,Category,Parameter,Value
0,Solar_Plant,unit,fom_cost,1.29
1,Electrolyzer,unit,fom_cost,4.34
2,Methanol_Reactor,unit,fom_cost,4.45
3,Steam_Plant,unit,fom_cost,0.119292
4,Methanol_Reactor,unit,min_down_time,48h


In [165]:
# Create a new data frame for parameters that are given as durations
# Necessary as SpineToolbox needs a separate input to map the parameter correctly

duration_parameter = 'min_down_time'
unit_parameters_duration_df = unit_parameters_df[unit_parameters_df['Parameter'] == duration_parameter]
# Resetting the index
unit_parameters_duration_df = unit_parameters_duration_df.reset_index(drop=True)

# Creating another DataFrame with rows that do not meet the condition
unit_parameters_rest_df = unit_parameters_df[unit_parameters_df['Parameter'] != duration_parameter]

In [167]:
# Create the balance type of the nodes
columns_to_select = ['Input1', 'Input2', 'Output1', 'Output2']
df_combined = pd.concat([df_model_units, df_model_connections])
df_combined = df_combined.reset_index(drop=True)

df_nodes_network = create_connection_dataframe(df_combined, columns_to_select)

# Get unique values from the 'in' column
unique_in_values = df_nodes_network['in'].dropna().unique()

# Identify values in 'in' column not present in 'out' column
values_not_in_out = unique_in_values[~pd.Series(unique_in_values).isin(df_nodes_network['out'].dropna().unique())]

# Get unique values from the 'in' column
unique_out_values = df_nodes_network['out'].dropna().unique()

# Identify values in 'in' column not present in 'out' column
values_not_in_in = unique_out_values[~pd.Series(unique_out_values).isin(df_nodes_network['in'].dropna().unique())]

# Create list of unique nodes that are either start or end nodes
unique_nodes = values_not_in_out.tolist() + values_not_in_in.tolist()
unique_nodes

df_nodes_network.replace(np.nan, None, inplace=True)

# Check for combinations that are mirrored
mirrored_combinations = find_mirror_combinations(df_nodes_network)

# Get information of connections for each node
partners_dict1 = find_partners(df_nodes_network)
partners_dict2 = find_partners(mirrored_combinations)

# Check both lists if there are identical entries and list nodes that only have a connection to the same node
# Storages are removed as they must be balanced
nodes_identical = find_identical_entries(partners_dict1, partners_dict2)

# Combined list of start and end nodes that should be unbalanced
unbalanced_nodes = nodes_identical + unique_nodes

df_nodes['balance_type'] = 'balance_type_node'
df_nodes.loc[df_nodes['Object_name'].isin(unbalanced_nodes), 'balance_type'] = 'balance_type_none'

In [169]:
# Add has_state_node_state_cap and frac_state_loss
df_storages_short = df_model_storages.loc[:, ['Storage', 'has_state', 'node_state_cap', 'frac_state_loss']].rename(columns={'Storage': 'Object_name'})
df_storages_short['has_state'] = df_storages_short['has_state'].astype(str).str.lower().replace('true', 'true')

df_nodes = pd.merge(df_nodes, df_storages_short, on='Object_name', how='left')
df_nodes['demand'] = ""

In [171]:
# Create a dataframe with all nodes that should have a slack based on the unit input and outputs
nodes_for_slack_df = check_entries_exist(df_model_units, 'node')
# Merge the information into the prepared data frame
merged_df = pd.merge(df_nodes, nodes_for_slack_df, left_on='Object_name', right_on='node', how='left')
merged_df = merged_df.drop(columns=['node'])
merged_df['node_slack_penalty'] = merged_df['node_slack_penalty'].replace({True: 1000000, False: ''})
# Create a dataframe with all nodes that should have a slack based on the connection input and outputs
nodes_for_slack_df2 = check_entries_exist(df_model_connections, 'connection')
# Merge the information into the prepared data frame
merged_df2 = pd.merge(df_nodes, nodes_for_slack_df2, left_on='Object_name', right_on='connection', how='left')
merged_df2 = merged_df2.drop(columns=['connection'])
merged_df2['node_slack_penalty'] = merged_df2['node_slack_penalty'].replace({True: 1000000, False: ''})
# Link the information about the slack of both data frames
merged_df2['node_slack_penalty'] = merged_df['node_slack_penalty'].combine_first(merged_df2['node_slack_penalty'])
# Clean the information that only nodes with 'balance_type_node' have a penalty
merged_df2.loc[merged_df2['balance_type'] == 'balance_type_none', 'node_slack_penalty'] = ''

# Add the information into the df_nodes
df_nodes['node_slack_penalty'] = merged_df2['node_slack_penalty']

# Show table head for control
df_nodes.head()

,Object_name,Category,balance_type,has_state,node_state_cap,frac_state_loss,demand,node_slack_penalty
0,Water,node,balance_type_none,NaN,NaN,NaN,,
1,Power_Kasso,node,balance_type_node,NaN,NaN,NaN,,1000000
2,Vaporized_Carbon_Dioxide,node,balance_type_node,NaN,NaN,NaN,,1000000
3,Hydrogen_storage_Kasso,node,balance_type_node,true,5478.676451,0.043264,,1000000
4,Carbon_Dioxide,node,balance_type_none,NaN,NaN,NaN,,


#### Times series:

In [174]:
# Adjust renewables columns names
df_PV_availabilityfactors_values.rename(columns={'time': 'time [UTC]', 
                                                 'local_time': 'time [' + area + ']',
                                                 'electricity': 'unit_availability_factor'}, inplace=True)
df_wind_availabilityfactors_values.rename(columns={'time': 'time [UTC]',
                                                   'local_time': 'time [' + area + ']',
                                                   'electricity_onshore': 'unit_availability_factor_onshore',
                                                   'electricity_offshore': 'unit_availability_factor_offshore'}, inplace=True)
# Adjust power prices
df_powerprices_values.rename(columns={'HourUTC': 'time [UTC]', 
                                         'HourDK': 'time [' + area + ']'}, inplace=True)

## Fitting data into format

### Relationships:

#### Object__from/to_node:

In [179]:
### UNITS ###
df_unit_relation_parameter_data = pd.DataFrame(object_relationship_unit_nodes(df_model_units))

In [181]:
# Add additional electricity connections if not already existent

units = df_units.iloc[:, 0].tolist()
length = len(units)
data = {
    "Relationship_class_name": ["unit__from_node"] * length,
    "Object_class": ["unit"] * length,
    "Object_name": units,
    "Node": ["Power_Kasso"] * length
}
df_electricity = pd.DataFrame(data)

units_with_from_node = df_unit_relation_parameter_data[df_unit_relation_parameter_data['Relationship_class_name'].str.contains('unit__from_node')]
valid_object_names = units_with_from_node['Object_name']
df_electricity_filtered = df_electricity[df_electricity['Object_name'].isin(valid_object_names)]

merged_df = pd.merge(df_electricity_filtered, units_with_from_node, on=['Relationship_class_name', 'Object_class', 'Object_name', 'Node'], how='left', indicator=True)
df_electricity_nodes = merged_df[merged_df['_merge'] == 'left_only'].drop(columns='_merge')

df_unit_relation_parameter_data = pd.concat([df_unit_relation_parameter_data, df_electricity_nodes], ignore_index=True)

In [183]:
### CONNECTIONS ###
df_connection_relation_parameter_data = object_relationship_connection_nodes(df_model_connections)

In [185]:
# Create combined DataFrame:
df_object__node = pd.concat([df_unit_relation_parameter_data, df_connection_relation_parameter_data])
df_object__node = df_object__node.reset_index(drop=True)

# Show df head for control
df_object__node.head()

,Relationship_class_name,Object_class,Object_name,Node,Parameter,Value
0,unit__to_node,unit,Solar_Plant,Power_Kasso,unit_capacity,100.0
1,unit__from_node,unit,Electrolyzer,Power_Kasso,unit_capacity,30.0
2,unit__from_node,unit,Electrolyzer,Power_Kasso,minimum_operating_point,0.02
3,unit__to_node,unit,Electrolyzer,Hydrogen_Kasso,,
4,unit__from_node,unit,Electrolyzer,Water,,


In [187]:
# Create a DataFrame for the definition of the object_node relationships
df_object__node_definitions = pd.DataFrame(df_object__node[['Relationship_class_name', 'Object_class', 'Object_name', 'Node']])
df_object__node_definitions = df_object__node_definitions.drop_duplicates()
df_object__node_definitions = df_object__node_definitions.reset_index(drop=True)

# To avoid import errors the connections  will be removed from the definitions df
# Dropping rows where 'Object_class' is 'connection'
df_object__node_definitions  = df_object__node_definitions [df_object__node_definitions ['Object_class'] != 'connection']

# Drop rows where no parameters for the relationship are defined (column has missing values (NaN or None))
drop_no_value_column = 'Parameter'
df_object__node_values = df_object__node[df_object__node[drop_no_value_column] != '']

# Show df head for control
df_object__node_definitions.head()

,Relationship_class_name,Object_class,Object_name,Node
0,unit__to_node,unit,Solar_Plant,Power_Kasso
1,unit__from_node,unit,Electrolyzer,Power_Kasso
2,unit__to_node,unit,Electrolyzer,Hydrogen_Kasso
3,unit__from_node,unit,Electrolyzer,Water
4,unit__to_node,unit,Electrolyzer,Waste_Heat


#### Object__node_node:

In [190]:
# Define which columns to check
columns_In_In_Unit = ['Unit', 'Input1', 'Input2']
columns_In_Out_Unit = ['Unit', 'Input1', 'Output1']
columns_Out_Out_Unit = ['Unit', 'Output1', 'Output2']
columns_In_In_Connection = ['Connection', 'Input1', 'Input2']
columns_In_Out_Connection = ['Connection', 'Input1', 'Output1']
columns_Out_Out_Connection = ['Connection', 'Output1', 'Output2']
columns_Out_In_Connection = ['Connection', 'Output1', 'Input1']

### UNITS ###
# Create list of tuples with values of cells + fix_ratio_XXX_XXX
values_in_in_units = [('unit__node__node', 'unit', row[columns_In_In_Unit[0]], row[columns_In_In_Unit[1]], row[columns_In_In_Unit[2]], 
                       'fix_ratio_in_in_unit_flow', row['Relation_In_In']) 
                      if not pd.isnull(row[columns_In_In_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_In_In_Unit]).any()]
values_in_out_units = [('unit__node__node', 'unit', row[columns_In_Out_Unit[0]], row[columns_In_Out_Unit[1]], row[columns_In_Out_Unit[2]], 
                        'fix_ratio_in_out_unit_flow', row['Relation_In_Out']) 
                      if not pd.isnull(row[columns_In_Out_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_In_Out_Unit]).any()]
values_out_out_units = [('unit__node__node', 'unit', row[columns_Out_Out_Unit[0]], row[columns_Out_Out_Unit[1]], row[columns_Out_Out_Unit[2]], 
                         'fix_ratio_out_out_unit_flow', row['Relation_Out_Out']) 
                      if not pd.isnull(row[columns_Out_Out_Unit]).any() else (np.nan, np.nan, None) for _, row in df_model_units.iterrows() 
                      if not pd.isnull(row[columns_Out_Out_Unit]).any()]

df_fix_ratio_in_in_units = pd.DataFrame(values_in_in_units, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                     'Node2', 'Parameter', 'Value'])
df_fix_ratio_in_out_units = pd.DataFrame(values_in_out_units, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                       'Node2', 'Parameter', 'Value'])
df_fix_ratio_out_out_units = pd.DataFrame(values_out_out_units, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                         'Node2', 'Parameter', 'Value'])

### CONNECTIONS ###
values_in_in_connections = [('connection__node__node', 'connection', row[columns_In_In_Connection[0]], 
                             row[columns_In_In_Connection[1]], row[columns_In_In_Connection[2]], 
                             'fix_ratio_in_in_connection_flow', row['Relation_In_In']) 
                            if not pd.isnull(row[columns_In_In_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                            if not pd.isnull(row[columns_In_In_Connection]).any()]
# Comment: connections_in_out are no longer supported by Spine, therefore the values get inverted
values_in_out_connections_inverted = [('connection__node__node', 'connection', row[columns_In_Out_Connection[0]], 
                              row[columns_In_Out_Connection[1]], row[columns_In_Out_Connection[2]], 
                              'fix_ratio_out_in_connection_flow', 1/row['Relation_In_Out']) 
                             if not pd.isnull(row[columns_In_Out_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                             if not pd.isnull(row[columns_In_Out_Connection]).any()]
values_out_out_connections = [('connection__node__node', 'connection', row[columns_Out_Out_Connection[0]], 
                               row[columns_Out_Out_Connection[1]], row[columns_Out_Out_Connection[2]], 
                               'fix_ratio_out_out_connection_flow', row['Relation_Out_Out']) 
                              if not pd.isnull(row[columns_Out_Out_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                              if not pd.isnull(row[columns_Out_Out_Connection]).any()]
values_out_in_connections = [('connection__node__node', 'connection', row[columns_Out_In_Connection[0]], 
                               row[columns_Out_In_Connection[1]], row[columns_Out_In_Connection[2]], 
                               'fix_ratio_out_in_connection_flow', row['Relation_Out_In']) 
                              if not pd.isnull(row[columns_Out_In_Connection]).any() else (np.nan, np.nan, None) for _, row in df_model_connections.iterrows() 
                              if not pd.isnull(row[columns_Out_In_Connection]).any()]

df_fix_ratio_in_in_connections = pd.DataFrame(values_in_in_connections, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                                 'Node2', 'Parameter', 'Value'])
df_fix_ratio_in_out_connections = pd.DataFrame(values_in_out_connections_inverted, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                                   'Node2', 'Parameter', 'Value'])
df_fix_ratio_out_out_connections = pd.DataFrame(values_out_out_connections, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                                     'Node2', 'Parameter', 'Value'])
df_fix_ratio_out_in_connections = pd.DataFrame(values_out_in_connections, columns=['Relationship', 'Object_class', 'Object_name', 'Node1', 
                                                                                     'Node2', 'Parameter', 'Value'])

# Create Object_node_node
df_object_node_node = pd.concat([df_fix_ratio_in_in_units, df_fix_ratio_in_out_units, df_fix_ratio_out_out_units, 
                                df_fix_ratio_in_in_connections, df_fix_ratio_in_out_connections, 
                                df_fix_ratio_out_out_connections, df_fix_ratio_out_in_connections])
df_object_node_node = df_object_node_node.reset_index(drop=True)

df_object_node_node = df_object_node_node.dropna(subset=['Value'])

# Show table head for control
df_object_node_node.head()

,Relationship,Object_class,Object_name,Node1,Node2,Parameter,Value
0,unit__node__node,unit,Electrolyzer,Power_Kasso,Water,fix_ratio_in_in_unit_flow,0.005850
1,unit__node__node,unit,CO2_Vaporizer,Power_Kasso,Carbon_Dioxide,fix_ratio_in_in_unit_flow,0.003601
2,unit__node__node,unit,Destilation_Tower,Raw_Methanol,Steam,fix_ratio_in_in_unit_flow,17.277902
3,unit__node__node,unit,Methanol_Reactor,Hydrogen_Kasso,Vaporized_Carbon_Dioxide,fix_ratio_in_in_unit_flow,5.173497
4,unit__node__node,unit,Steam_Plant,Power_Kasso,Water,fix_ratio_in_in_unit_flow,0.000724


In [192]:
# For storages, the out_in_connetion is set in both directions

# Step 1: Check if the 'Object_name' column contains the word 'storage'
storage_condition = df_object_node_node['Object_name'].str.contains('storage', case=False)

# Step 2: Check if the 'Parameter' column starts with 'fix_ratio_'
parameter_condition = df_object_node_node['Parameter'].str.startswith('fix_ratio_')

# Step 3: Combine both conditions
combined_condition = storage_condition & parameter_condition

# Step 2: Update the 'Parameter' column for rows that meet the condition
df_object_node_node.loc[storage_condition, 'Parameter'] = 'fix_ratio_out_in_connection_flow'

In [194]:
# Check if second node is necessary for demand
for index, row in df_model_units.iterrows():
    df_definition, df_nodes, df_connections, df_object__node_values, df_object_node_node = check_demand_node(
        row, temporal_block, resolution_to_block, df_definition, 
        df_nodes, df_connections, df_object__node_values, 
        df_object_node_node)

#### Hard-coded parameter for destillation tower (please remove once better solution is found)

In [197]:
data = {
    "Relationship": ["unit__node__node"],
    "Object_class": "unit",
    "Object_name": "Destilation_Tower",
    "Node1": "Power_Kasso",
    "Node2": "E-Methanol_Kasso",
    "Parameter": "max_ratio_in_out_unit_flow",
    "Value": 0.000001
}
df_hardcode_destillation = pd.DataFrame(data)

# Append new rows to the original DataFrame
df_object_node_node = pd.concat([df_object_node_node, df_hardcode_destillation], ignore_index=True)

#### Investments:

In [200]:
# Initialize df
'''TODO: Find a way to work with the existing capacity as share in the number of units but regarding the new
way of using a investment limit (there must be a change regarding the current implementation. 
In this case, the unit_capacity must be changed to the investment_limit.'''

df_units_inv_parameters = pd.DataFrame(columns=['Object_name', 'unit_investment_variable_type', 
                                                    'initial_units_invested_available', 'number_of_units', 
                                                    'candidate_units', 'unit_investment_cost', 
                                                    'unit_investment_tech_lifetime', 'unit_investment_econ_lifetime'])

# Choose correct column for investment costs based on chosen year
year_columns = {int(col.split()[-1]): col for col in df_investment_costs_raw.columns if 'Value' in col}
closest_year = min(year_columns.keys(), key=lambda x: abs(x - year))
selected_column = year_columns[closest_year]

# Add values if investment is selected
if candidate_nonzero:
    rows_to_add = []
    
    for index, row in df_model_units_raw.iterrows():
        object_type = row['Object_type']
        
        cap_input = row['Cap_Input1_existing'] if not pd.isna(row['Cap_Input1_existing']) else 0
        cap_output = row['Cap_Output1_existing'] if not pd.isna(row['Cap_Output1_existing']) else 0
        
        # Default investment cost
        matching_row = df_investment_costs_raw[df_investment_costs_raw['Object_type'] == object_type]
        if not matching_row.empty:
            # Extract the investment cost and lifetime from matching_row
            investment_cost = matching_row[selected_column].values[0]
            lifetime_str = matching_row['Lifetime'].values[0]
            
            # Convert the lifetime to days
            lifetime = convert_to_days(lifetime_str, year)
        else:
            investment_cost = 0  # Default investment cost if no match is found
            lifetime = '10950D'  # Default lifetime if no match is found
        default_value = investment_cost * (cap_input + cap_output)
        
        # Externally chosen investment cost
        pattern = re.compile(rf"inv_cost_.*{object_type}")
        matching_vars = [var for var in globals() if pattern.match(var)]
        
        if matching_vars:
            variable_name = matching_vars[0]
            variable_value = globals()[variable_name]
            
            if variable_value is not None:
                special_value = variable_value * (cap_input + cap_output)
                unit_investment_cost = special_value
            else:
                unit_investment_cost = default_value
        else:
            # If no external costs, use the default cost
            unit_investment_cost = default_value
        
        # Add the results for this row to df_units_inv_parameters
        row_to_add = {'Object_name': row['Unit'],
            'unit_investment_variable_type': 'unit_investment_variable_type_continuous',
            'initial_units_invested_available': 0,
            'number_of_units': row['number_of_units'],
            #new way, when using exisiting capacities
            #'number_of_units': EXISITING / LIMIT,
            'candidate_units': 1,
            'unit_investment_cost': unit_investment_cost,
            #new way, when using exisiting capacities
            #'unit_investment_cost': LIMIT * INV_COST_MW,
            'unit_investment_tech_lifetime': lifetime,
            'unit_investment_econ_lifetime': lifetime  # Same as tech lifetime
        }
        
        rows_to_add.append(row_to_add)
        
    df_units_inv_parameters = pd.concat([df_units_inv_parameters, pd.DataFrame(rows_to_add)], ignore_index=True)
    
    # Scale investment costs to lifetime
    df_units_inv_parameters = scale_costs(df_units_inv_parameters, start_date, end_date, 'unit')
    
# Show table head for control
df_units_inv_parameters.head()

C:\Users\jfg.eco\AppData\Local\Temp\ipykernel_24248\1821009392.py:74: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_units_inv_parameters = pd.concat([df_units_inv_parameters, pd.DataFrame(rows_to_add)], ignore_index=True)


,Object_name,unit_investment_variable_type,initial_units_invested_available,number_of_units,candidate_units,unit_investment_cost,unit_investment_tech_lifetime,unit_investment_econ_lifetime
0,Solar_Plant,unit_investment_variable_type_continuous,0,0,1,1.595616e+06,12775D,12775D
1,Electrolyzer,unit_investment_variable_type_continuous,0,0,1,2.273753e+06,9125D,9125D
2,CO2_Vaporizer,unit_investment_variable_type_continuous,0,0,1,2.493151e+06,7300D,7300D
3,Destilation_Tower,unit_investment_variable_type_continuous,0,0,1,2.333589e+06,10950D,10950D
4,Methanol_Reactor,unit_investment_variable_type_continuous,0,0,1,2.333589e+06,10950D,10950D


In [202]:
#function to filter the investment parameters for later use
#this must be improved to integrate all RES into the dataframe not just solar
df_units_inv_parameters_test = update_units_inv_parameters(df_units_inv_parameters, ['Solar_Plant'], candidate_nonzero)
df_units_inv_parameters_test

,Object_name,unit_investment_variable_type,initial_units_invested_available,number_of_units,candidate_units,unit_investment_cost,unit_investment_tech_lifetime,unit_investment_econ_lifetime
0,Solar_Plant,unit_investment_variable_type_continuous,0,1,1,1.595616e+06,12775D,12775D
1,Electrolyzer,unit_investment_variable_type_continuous,0,0,1,2.273753e+06,9125D,9125D
2,CO2_Vaporizer,unit_investment_variable_type_continuous,0,0,1,2.493151e+06,7300D,7300D
3,Destilation_Tower,unit_investment_variable_type_continuous,0,0,1,2.333589e+06,10950D,10950D
4,Methanol_Reactor,unit_investment_variable_type_continuous,0,0,1,2.333589e+06,10950D,10950D
5,Steam_Plant,unit_investment_variable_type_continuous,0,0,1,7.479452e+05,7300D,7300D


In [204]:
df_investment_params

,Object_name,investment_cost,investment_limit,capacities_exisiting
0,Electrolyzer,100.0,NaN,30.0
1,Hydrogen_storage_Kasso,100.0,NaN,NaN
2,Methanol_Reactor,100.0,NaN,NaN
3,E-Methanol_storage_Kasso,10.0,NaN,NaN
4,Steam_Plant,100.0,NaN,NaN


In [ ]:
#for testing
df_investment_params['investment_limit'] = 10
#relevant for later operation
df_investment_params['investment_cost_total'] = df_investment_params['investment_limit']*df_investment_params['investment_cost']
df_investment_params

In [ ]:
#adjust the investment values in case there are values defined by the user request
#necessary to change the number_of_units in case of defined existing capacities
#necessary to adjust the unit_capacity in case of new investment limit
#necessary to change the unit_investment_cost in case of new investment limit


#changes in the df_units_inv_parameters for number_of_units and unit_investment_cost
df_units_inv_parameters

#changes in the df_object__node of the unit_capacity
#relevant to change the investment cells to a later point in the notebook
df_object__node

In [ ]:
# Initialize df
df_connections_inv_parameters = pd.DataFrame(columns=['Object_name', 'Parameter_name',
                                                      'Connection_type', 'connection_investment_variable_type',
                                                      'initial_connections_invested_available', 'number_of_connections',
                                                      'candidate_connections', 'connection_investment_cost',
                                                      'connection_investment_tech_lifetime', 'connection_investment_econ_lifetime'])

# Add values if investment is selected
if candidate_nonzero:
    rows_to_add = []
    
    for index, row in df_model_connections_raw.iterrows():
        object_type = row['Object_type']
        
        matching_row = df_investment_costs_raw[df_investment_costs_raw['Object_type'] == object_type]
        if not matching_row.empty:
            # Extract the investment cost and lifetime from matching_row
            investment_cost_default = matching_row[selected_column].values[0]
        else:
            investment_cost_default = 0  # Default investment cost if no match is found
        
        # Scale default costs up with meters
        
        connection_type = row['Connection_type']
        lifetime = row['connection_investment_tech_lifetime']
        lifetime_str = row['connection_investment_tech_lifetime']
        # Convert the lifetime to days
        lifetime = convert_to_days(lifetime_str, year)
        
        # Add the results for this row to df_units_inv_parameters
        row_to_add = {'Object_name': row['Connection'],
                      'Parameter_name': 'connection_type',
                      'Connection_type': connection_type,
                      'connection_investment_variable_type': 'connection_investment_variable_type_continuous',
                      'initial_connections_invested_available': 0,
                      'number_of_connections': row['number_of_connections'],
                      'candidate_connections': 1,
                      'connection_investment_cost': investment_cost_default,
                      'connection_investment_tech_lifetime': lifetime,
                      'connection_investment_econ_lifetime': lifetime  # Same as tech lifetime
        }
        
        rows_to_add.append(row_to_add)
        
    df_connections_inv_parameters = pd.concat([df_connections_inv_parameters, pd.DataFrame(rows_to_add)], ignore_index=True)
    
    # Scale investment costs to lifetime
    df_connections_inv_parameters = scale_costs(df_connections_inv_parameters, start_date, end_date, 'connection')
    
# Show table head for control
df_connections_inv_parameters.head()

In [ ]:
# Initialize df
df_storages_inv_parameters = pd.DataFrame(columns=['Object_name', 'storage_investment_variable_type',
                                                       'initial_storages_invested', 'number_of_storages',
                                                       'candidate_storages', 'storage_investment_cost',
                                                       'storage_investment_tech_lifetime', 'storage_investment_econ_lifetime'])

# Add values if investment is selected
if candidate_nonzero:
    rows_to_add = []
    
    for index, row in df_model_storages_raw.iterrows():
        object_type = row['Object_type']
        
        node_value = row['node_state_cap'] if not pd.isna(row['node_state_cap']) else 0
        
        # Default investment cost
        matching_row = df_investment_costs_raw[df_investment_costs_raw['Object_type'] == object_type]
        if not matching_row.empty:
            # Extract the investment cost and lifetime from matching_row
            investment_cost = matching_row[selected_column].values[0]
            lifetime_str = matching_row['Lifetime'].values[0]
            # Convert the lifetime to days
            lifetime = convert_to_days(lifetime_str, year)
            
        else:
            investment_cost = 0  # Default investment cost if no match is found
            lifetime = '10950D'  # Default lifetime if no match is found
        
        default_value = investment_cost * node_value
        
        # Externally chosen investment cost
        pattern = re.compile(rf"inv_cost_.*{object_type}")
        matching_vars = [var for var in globals() if pattern.match(var)]
        
        if matching_vars:
            variable_name = matching_vars[0]
            variable_value = globals()[variable_name]
            
            if variable_value is not None:
                special_value = variable_value * node_value
                storage_investment_cost = special_value
            else:
                storage_investment_cost = default_value
        else:
            # If no matching variable, use the default calculation
            storage_investment_cost = default_value
        
        # Add the results for this row to df_units_inv_parameters
        row_to_add = {'Object_name': row['Storage'],
            'storage_investment_variable_type': 'storage_investment_variable_type_continuous',
            'initial_storages_invested': 0,
            'number_of_storages': row['number_of_storages'],
            'candidate_storages': 1,
            'storage_investment_cost': storage_investment_cost,
            'storage_investment_tech_lifetime': lifetime,
            'storage_investment_econ_lifetime': lifetime  # Same as tech lifetime
        }
        
        rows_to_add.append(row_to_add)
        
    df_storages_inv_parameters = pd.concat([df_storages_inv_parameters, pd.DataFrame(rows_to_add)], ignore_index=True)
    
    # Scale investment costs to lifetime
    df_storages_inv_parameters = scale_costs(df_storages_inv_parameters, start_date, end_date, 'storage')
    
# Add to df_nodes table
df_nodes = pd.merge(df_nodes, df_storages_inv_parameters, on = 'Object_name', how = 'left')

### Model:

#### Model relations:

In [ ]:
# Writing definition of model components
column_names_model_components = {'Object_class_name':['model','temporal_block','stochastic_scenario', 'stochastic_structure', 'report'],
                      'Object_name': [model_name, temporal_block, stochastic_scenario, stochastic_structure, report_name]}
df_model_components = pd.DataFrame(column_names_model_components, index=None)

# Adding a default investment temporal block if investment = true 
if candidate_nonzero:
    df_model_components.loc[len(df_model_components.index)] = ['temporal_block', 'Default_Investment_period']

# Outputs:
reports_list = list(reports)
df_outputs = pd.DataFrame({
    'Object_class_name': ['output']*len(reports_list),
    'Object_name': reports_list
})
df_model_components = pd.concat([df_model_components, df_outputs], axis=0)
df_model_components = df_model_components.reset_index(drop=True)

In [ ]:
# Add reports chosen in main
column_names_model_structure = {'Object_class_name':['model','temporal_block','stochastic_scenario', 'stochastic_structure', 'report'],
                      'Object_name': [model_name, temporal_block, stochastic_scenario, stochastic_structure, report_name]}
df_reports = pd.DataFrame({
    'Relationship_class_name': ['report__output']*len(reports_list),
    'Object_class_name_1': ['report']*len(reports_list),
    'Object_class_name_2': ['output']*len(reports_list),
    'Object_name_1': [report_name]*len(reports_list),
    'Object_name_2': reports_list
})

# Add everything else
df_model_struc = pd.DataFrame({
    'Relationship_class_name': ['model__temporal_block','model__default_temporal_block', 
                                'model__stochastic_structure', 'model__default_stochastic_structure',
                                'stochastic_structure__stochastic_scenario', 'model__report'],
    'Object_class_name_1': ['model','model','model','model','stochastic_structure','model'],
    'Object_class_name_2': ['temporal_block', 'temporal_block','stochastic_structure','stochastic_structure', 'stochastic_scenario','report'],
    'Object_name_1': [model_name, model_name,model_name,model_name,stochastic_structure, model_name],
    'Object_name_2': [temporal_block, temporal_block, stochastic_structure, stochastic_structure, stochastic_scenario, report_name]
})
df_model_relations = pd.concat([df_model_struc, df_reports], axis=0)
df_model_relations = df_model_relations.reset_index(drop=True)

# Defining default investment temporal block and stochastic structure if investment = true
if candidate_nonzero:
    df_model_relations.loc[len(df_model_relations.index)] = ['model__default_investment_temporal_block', 'model', 'temporal_block', model_name, 'Default_Investment_period']
    df_model_relations.loc[len(df_model_relations.index)] = ['model__default_investment_stochastic_structure', 'model', 'stochastic_structure', model_name, stochastic_structure]

# Show table head for control
df_model_relations.head()

#### Start, end, resolution :

In [ ]:
# Create values for model structure
column_names_model = {'Object_class_name':['model','model','temporal_block'],
                      'Object_name': [model_name, model_name, temporal_block],
                      'Parameter':['model_start','model_end','resolution'],
                      'Alternative': [scenario[0], scenario[0], scenario[0]],
                      'Value': ['{"type": "date_time", "data": "'+df_time.iloc[0]['DateTime']+'"}',
                          '{"type": "date_time", "data": "'+df_time.iloc[-1]['DateTime']+'"}', 
                          '{"type":"duration", "data": "'+frequency+'"}']}
df_model = pd.DataFrame(column_names_model, index=None)

# Add investment temporal blocks to model data frame in days if investment = true
if candidate_nonzero:
    df_model.loc[len(df_model.index)] = ['temporal_block', 'Default_Investment_period', 'resolution', scenario[0], '{"type":"duration", "data": "'+convert_to_days(investment_period_default, year)+'"}']

# Show how table head for control
df_model.head()

In [ ]:
# Add information in case roll forward is used
if roll_forward_use:
    if temporal_block == 'hourly':
        unit = 'h'
    elif temporal_block == 'daily':
        unit = 'D'
    elif temporal_block == 'monthly':
        unit = 'M'
    else:
        unit = ''
        print("\033[91mWARNING:\033[0m Duration not defined!!!")
    # Add the new row to the df
    roll_forward_size_with_unit = str(roll_forward_size) + unit
    roll_forward_row = {'Object_class_name': 'model', 
                        'Object_name': model_name, 
                        'Parameter': 'roll_forward',
                        'Alternative': scenario[0],
                        'Value': '{"type": "duration", "data": "' + roll_forward_size_with_unit +'"}'
                       }
    # Add new row to df_model DataFrame
    df_model.loc[len(df_model)] = roll_forward_row

# Show table head for control
df_model.head()

In [ ]:
# Temporal resolution for demand
# Add temporal block to definition, if necessary
df_model_units['resolution_output'].apply(lambda x: check_temporal_block(x, df_model_components))

# Add temporal relation to respective node
df_temporal_relations = pd.DataFrame(columns = ['Relationship_class_name', 'Node', 'Temporal_block'], index=None)
df_model_units.apply(lambda row: create_temporal_block_relationships(row['resolution_output'],row['Output1'], df_model_relations, model_name, df_definition, df_temporal_relations),axis=1)

# Add values to temporal block
df_model_units['resolution_output'].apply(lambda x: create_temporal_block_input(x, df_model))

### Variable Efficiency Units:

#### Calculate adjusted efficiency for each unit

In [ ]:
# Important as the SpineOpt implementation is piecewise but when producing at high capacity the lower segments also produce
# This way, we avoid overestimation of production when running at higher capacities

for index, row in df_model_units.iterrows():
    if pd.notna(row['mean_efficiency']):
        # If 'mean_efficiency' column has a value, check if corresponding DataFrame exists
        column_name = row['Unit'].lower()
        df_name = f"df_efficiency_{column_name}"
        goal = row['mean_efficiency']
        output_1 = row['Output1']
        input_1 = row['Input1']
        
        if df_name in globals() and isinstance(globals()[df_name], pd.DataFrame):
            
            # Calculate adjusted efficiency
            df_efficiency_adj = create_adj_efficiency(globals()[df_name], goal, column_name)
            globals()[f"df_efficiency_{column_name}_adj"] = df_efficiency_adj
            
            # Fit with operating points
            des_segment = globals()['des_segments_' + column_name]
            df_var_efficiency, df_operating_points, segment_x_values, segment_averages, x_values, y_values = calculate_op_points(
                column_name, des_segment, df_efficiency_adj, input_1, output_1
            )
            globals()[f"variable_efficiency_{column_name}"] = df_var_efficiency
            globals()[f"operating_points_{column_name}"] = df_operating_points
            globals()[f"segment_x_values_{column_name}"] = segment_x_values
            globals()[f"segment_averages_{column_name}"] = segment_averages
            globals()[f"x_values_{column_name}"] = x_values
            globals()[f"y_values_{column_name}"] = y_values
            
            # Create ordered_unit_flow
            ordered_unit_flow = check_decreasing(df_var_efficiency, column_name, input_1)
            globals()[f"ordered_unit_flow_{column_name}"] = ordered_unit_flow
            
        else:
            print(f"WARNING: No variable efficiency defined for {column_name}")
    else:
        # If 'mean_efficiency' column is NA, skip
        continue

In [ ]:
# Plot the data of the original efficiency curve and the adjusted ones
plt.figure(figsize=(7, 3))
plt.plot(df_efficiency_electrolyzer_adj['Power [%]'], df_efficiency_electrolyzer_adj['Efficiency [%]'], label='Eff raw', marker='o')
plt.plot(df_efficiency_electrolyzer_adj['Power [%]'], df_efficiency_electrolyzer_adj['Efficiency_scaled [%]'], label='Eff scaled', marker='x')
plt.plot(df_efficiency_adj['Power [%]'], df_efficiency_adj['eff_adjusted_electrolyzer'], label='Eff adjusted', marker='x', 
         linestyle='dashed', dashes=(5, 7))

plt.xlabel('Power [%]')
plt.ylabel('Efficiency [%]')
plt.title('Efficiency vs Power')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
# Plotting the data points and the curve
plt.figure(figsize=(5, 4))
plt.scatter(df_efficiency_electrolyzer_adj['Power [%]'], df_efficiency_electrolyzer_adj['eff_adjusted_electrolyzer'], color='blue', label='Efficiency Adjusted')
plt.plot(x_values_electrolyzer, y_values_electrolyzer, color='red', label='Fitted Efficiency Curve Electrolyzer')

# Plotting segment averages
for i, (x_start, x_end) in enumerate(segment_x_values_electrolyzer):
    plt.plot([x_start, x_end], [segment_averages_electrolyzer[i], segment_averages_electrolyzer[i]], color='green', linestyle='--', linewidth=2)

plt.xlabel('x')
plt.ylabel('y')
plt.title('Curve Fitted to Data Points')
plt.legend()
plt.grid(True)
plt.show()

#### Add operating points to user constraint

In [ ]:
###Summarize all variable efficiencies
# Collect all data frames that start with variable_efficiency
dfs = [value for key, value in globals().items() if key.startswith('variable_efficiency') and isinstance(value, pd.DataFrame)]

# Choose the first column with the most segments
longest_df = max(dfs, key=lambda df: df.shape[0])
first_column = longest_df.iloc[:, 0]

# Concatenate the remaining columns from all DataFrames, ensuring no duplicate "first column"
remaining_columns = [df.iloc[:, 1:] for df in dfs]

# Concatenate the remaining columns to the longest first column
df_variable_efficiency = pd.concat([first_column] + remaining_columns, axis=1)
df_variable_efficiency

In [ ]:
# Relate all user_constraints to entity class name "user_constraint" 

Entity_names_duplicate = df_variable_efficiency.iloc[0,1:]
User_constraint_entities = []
for name in Entity_names_duplicate:
    if name not in User_constraint_entities:
        User_constraint_entities.append(name)
        

User_constraint_column = ["user_constraint"]*len(User_constraint_entities)

merged_columns = {"Entity class names": User_constraint_column, "Entity names": User_constraint_entities}
df_variable_eff_def= pd.DataFrame(merged_columns)
df_variable_eff_def

In [ ]:
### Summarize all operating points
# Collect all data frames that start with operating_points
dfs = [value for key, value in globals().items() if key.startswith('operating_points') and isinstance(value, pd.DataFrame)]

# Choose the first column with the most segments
longest_df = max(dfs, key=lambda df: df.shape[0])
first_column = longest_df.iloc[:, 0]

# Concatenate the remaining columns from all DataFrames, ensuring no duplicate "first column"
remaining_columns = [df.iloc[:, 1:] for df in dfs]

# Concatenate the remaining columns to the longest first column
df_operating_points = pd.concat([first_column] + remaining_columns, axis=1)

In [ ]:
### Summarize all ordered_unit_flow_op
# Collect all data frames that start with ordered_unit_flow_op
dfs = [value for key, value in globals().items() if key.startswith('ordered_unit_flow_') and isinstance(value, pd.DataFrame)]

# Concatenate the remaining columns to the longest first column
df_boolean_relations = pd.concat(dfs, ignore_index=True)

### Environment and Storages

#### Renewables Availability:

In [ ]:
# Create table headers and relations
column_names_1 = {'DateTime '+area: [None, None]}

# Add time values
df_temp_1 = pd.DataFrame(columns=['DateTime ' + area])
df_temp_1['DateTime '+area] = df_time

# Add solar values
if 'Solar plant' in powers:
    column_names_1['Solar_Plant'] = ['unit', 'unit_availability_factor']
    df_temp_1['Solar_Plant'] = df_PV_availabilityfactors_values['unit_availability_factor']
    
    # Fill NaNs on last day by copying previous day (only for leap years)
    df_temp_1['Solar_Plant_shifted'] = df_temp_1['Solar_Plant'].shift(24)
    df_temp_1['Solar_Plant'] = df_temp_1['Solar_Plant'].fillna(df_temp_1['Solar_Plant_shifted'])
    df_temp_1 = df_temp_1.drop(columns=['Solar_Plant_shifted'])

# Add onshore wind values
if 'Wind onshore' in powers:
    column_names_1['Wind_onshore'] = ['unit', 'unit_availability_factor']
    df_temp_1['Wind_onshore'] = df_wind_availabilityfactors_values['unit_availability_factor_onshore']
    
    # Fill NaNs on last day by copying previous day (only for leap years)
    df_temp_1['Wind_onshore_shifted'] = df_temp_1['Wind_onshore'].shift(24)
    df_temp_1['Wind_onshore'] = df_temp_1['Wind_onshore'].fillna(df_temp_1['Wind_onshore_shifted'])
    df_temp_1 = df_temp_1.drop(columns=['Wind_onshore_shifted'])

# Add offshore wind values
if 'Wind offshore' in powers:
    column_names_1['Wind_offshore'] = ['unit', 'unit_availability_factor']
    df_temp_1['Wind_offshore'] = df_wind_availabilityfactors_values['unit_availability_factor_offshore']
    
    # Fill NaNs on last day by copying previous day (only for leap years)
    df_temp_1['Wind_offshore_shifted'] = df_temp_1['Wind_offshore'].shift(24)
    df_temp_1['Wind_offshore'] = df_temp_1['Wind_offshore'].fillna(df_temp_1['Wind_offshore_shifted'])
    df_temp_1 = df_temp_1.drop(columns=['Wind_offshore_shifted'])


#Add chosen availability factors to one combined dataframe
df_blank_table_1 = pd.DataFrame(column_names_1, index=None)
df_time_series = pd.concat([df_blank_table_1, df_temp_1])

# Show table head for control
df_time_series.head()

#### Energy prices:

In [ ]:
# Adjustments of power price variance

# Calculate the current mean and variance
mean = df_powerprices_values['SpotPriceEUR'].mean()
current_variance = df_powerprices_values['SpotPriceEUR'].var()

# Define the new desired variance 
desired_variance = current_variance * power_price_variance

# Calculate the scaling factor
scaling_factor = np.sqrt(desired_variance / current_variance)

# Adjust the time series to achieve the new variance
df_powerprices_values['SpotPriceEUR'] = mean + (df_powerprices_values['SpotPriceEUR'] - mean) * scaling_factor

In [ ]:
# Create table
column_names_2 = {'DateTime ' + area: ['relationship class','connection','node','parameter name'],
                'Power_Wholesale_In': ['connection__from_node','power_line_Wholesale_Kasso','Power_Wholesale','connection_flow_cost'], 
                'Power_Wholesale_Out': ['connection__to_node','power_line_Wholesale_Kasso','Power_Wholesale','connection_flow_cost'], 
                'District_Heating': ['connection__to_node','pipeline_District_Heating','District_Heating','connection_flow_cost']}
df_blank_table_2 = pd.DataFrame(column_names_2, index=None)

# Add values
df_temp_2 = pd.DataFrame(columns=['DateTime ' + area, 'Power_Wholesale_In', 'Power_Wholesale_Out', 'District_Heating'])

df_temp_2['DateTime ' + area] = df_time
df_temp_2['Power_Wholesale_In'] = price_level_power * df_powerprices_values['SpotPriceEUR']
df_temp_2['Power_Wholesale_Out'] = -1 * price_level_power * df_powerprices_values['SpotPriceEUR']
df_temp_2['District_Heating'] = -1 * df_district_heating_price[str(year)].loc[2] * share_of_dh_price_cap

df_table_2 = pd.concat([df_blank_table_2, df_temp_2], ignore_index=True)

In [ ]:
# Add grid costs for consumption and production
# Create table
column_names_3 = {'Grid_costs_consumption': ['connection__to_node', 'power_line_Wholesale_Kasso', 'Power_Kasso', 'connection_flow_cost'],
                 'Grid_costs_production': ['connection__from_node', 'power_line_Wholesale_Kasso', 'Power_Kasso', 'connection_flow_cost']}
df_blank_table_3 = pd.DataFrame(column_names_3, index=None)

# Add values
df_temp_3 = pd.DataFrame(columns=['Grid_costs_consumption', 'Grid_costs_production'])

df_temp_3['Grid_costs_consumption'] = df_grid_costs['consumption costs']
df_temp_3['Grid_costs_production'] = df_grid_costs['production costs']

df_table_3 = pd.concat([df_blank_table_3, df_temp_3], ignore_index = True)

# Merge all energy prices
df_energy_prices = pd.concat([df_table_2, df_table_3], axis = 1)

# Show table head for control
df_energy_prices.head()

#### Units on costs:

In [ ]:
# Add units on costs for all units that have either min_down_time, ramp_up/down_limits, or start_up/shut_down limits/costs
# Filter units_parameter_df and df_object__node
cost_parameters = ['min_down_time', 'min_up_time', 'shut_down_cost', 'start_up_cost', 
                   'ramp_down_limit', 'ramp_up_limit', 'start_up_limit', 'shut_down_limit']
filtered_units_parameter_df = unit_parameters_df[unit_parameters_df['Parameter'].isin(cost_parameters)]
filtered_df_object__node = df_object__node[df_object__node['Parameter'].isin(cost_parameters)]

combined_object_names = pd.concat([filtered_units_parameter_df['Object_name'], filtered_df_object__node['Object_name']])
unique_objects = combined_object_names.unique()

# Check if they already have a units_on_cost value other than 0
non_zero_units_df = unit_parameters_df[(unit_parameters_df['Parameter'] == 'units_on_cost') & 
                                       (unit_parameters_df['Value'] != 0.0)]
non_zero_units = non_zero_units_df['Object_name'].unique()

# Remove units_on_cost = 0 from unit_parameters_df if in unique_objects
unit_parameters_df = unit_parameters_df[~(
    (unit_parameters_df['Parameter'] == 'units_on_cost') & 
    (unit_parameters_df['Value'] == 0) & 
    (unit_parameters_df['Object_name'].isin(unique_objects))
)]

unique_units = [unit for unit in unique_objects if unit not in non_zero_units]

# Create new columns for each unique unit in df_time_series
for unit in unique_units:
    unit_data = ["unit", "units_on_cost"]
    
    power_data = df_temp_2['Power_Wholesale_In'].tolist()
    
    #df_time_series[unit] = unit_data + power_data
    # Reset the index of df_time_series to avoid non-unique index error
    df_time_series = df_time_series.reset_index(drop=True)
    # Create a temporary DataFrame for the new column
    new_column = pd.DataFrame({unit: unit_data + power_data})
    # Ensure the new column has the same length as df_time_series
    new_column = new_column.reindex(df_time_series.index)
    # Use pd.concat to add new columns without overwriting existing ones
    df_time_series = pd.concat([df_time_series, new_column], axis=1)


# Show table head for control
df_time_series.head()

#### Time Series Storage:

In [ ]:
# Date index
start_date = pd.to_datetime(start_date)
before = start_date-timedelta(hours=1)
date_index_beginning = pd.date_range(start=before, end=start_date, freq='H')
formatted_beginning = date_index_beginning.strftime('%Y-%m-%dT%H:%M:%S')
df_formatted_beginning = pd.DataFrame(formatted_beginning, columns=['DateTime'])
df_time_beginning = pd.DataFrame(df_formatted_beginning)
# Add one blank row
new_row = pd.Series([])
df_time_beginning = pd.concat([pd.DataFrame([new_row]), df_time_beginning]).reset_index(drop=True)

# Concat raw data with time index
storage_values = df_model_storages.iloc[:,[0, 1, 2, 6]]
storage_values = storage_values.iloc[:, [0, 3, 1, 2]]
storage_values_transposed = storage_values.T
storage_values_transposed.columns = storage_values_transposed.iloc[0]
storage_values_transposed = storage_values_transposed[1:]
storage_values_transposed.reset_index(drop=True, inplace=True)

df_storage = pd.concat([df_time_beginning, storage_values_transposed], axis=1)

# Show table head for control
df_storage.head()

## Creating one combined excel and export

In [ ]:
# Create the prepared input excel for the use in SpineToolbox
with pd.ExcelWriter(output_file_path + output_file_name) as writer:
    df_definition.to_excel(writer, sheet_name='Definition', index=False)
    unit_parameters_rest_df.to_excel(writer, sheet_name='Definition_parameters', index=False)
    unit_parameters_duration_df.to_excel(writer, sheet_name='Definition_parameters_duration', index=False)
    df_units_inv_parameters.to_excel(writer, sheet_name='Unit_Inv_Parameters', index=False)
    df_nodes.to_excel(writer, sheet_name='Nodes', index=False)
    df_connections_inv_parameters.to_excel(writer, sheet_name='Connection_Inv_Parameters', index=False)
    df_object__node_definitions.to_excel(writer, sheet_name='Object__to_from_node_definition', index=False)
    df_object__node_values.to_excel(writer, sheet_name='Object__to_from_node', index=False)
    df_boolean_relations.to_excel(writer, sheet_name='Boolean_relations', index=False)
    df_object_node_node.to_excel(writer, sheet_name='Object__node_node', index=False)
    df_variable_eff_def.to_excel(writer, sheet_name='Variable_Eff_Definition', index=False)
    df_variable_efficiency.to_excel(writer, sheet_name='Variable_Eff', index=False)
    df_operating_points.to_excel(writer, sheet_name='Operating_points', index=False)
    df_storage.to_excel(writer, sheet_name='Time_series_storage', index=False)
    df_time_series.to_excel(writer, sheet_name='Time_series', index=False)
    df_energy_prices.to_excel(writer, sheet_name='Energy_prices', index=False)
    df_model_components.to_excel(writer, sheet_name='Model_components', index=False)
    df_model_relations.to_excel(writer, sheet_name='Model_relations', index=False)
    df_model.to_excel(writer, sheet_name='Model', index=False)
    df_temporal_relations.to_excel(writer, sheet_name='Temporal_relations', index=False)

In [ ]:
# Create output of the mapping excel
with pd.ExcelWriter(output_file_path + output_mapping_file_name) as writer:
    df_model_object_mapping.to_excel(writer, sheet_name='Object_Mapping', index=False)

In [ ]:
# Copy efficiency
other_name = "other.xlsx"
with pd.ExcelWriter(output_file_path + other_name) as writer:
    df_efficiency_electrolyzer_adj.to_excel(writer, sheet_name='efficiency', index=False)